In [1]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_empirical_variable_cost_runs(acq, tfn):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOPT", filters={
        "sweep": "23nvpul1", 
        "config.problem": tfn,
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [2]:
acquisition_functions = ['BudgetedMultiStepLookaheadEI']
target_functions = ["LunarLander"]

In [4]:
grouped_runs = {(a,t): load_empirical_variable_cost_runs(a,t) for a in acquisition_functions for t in target_functions}

100%|██████████| 16/16 [00:16<00:00,  1.06s/it]


In [4]:
for a in acquisition_functions:
    for t in target_functions:
        config_and_metrics_per_seed = grouped_runs[a,t]

        cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
        best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T

        print(a, t, best_observed_per_seed.shape)
        
        best_25 = np.quantile(best_observed_per_seed, 0.25, axis=1)
        best_50 = np.quantile(best_observed_per_seed, 0.5, axis=1)
        best_75 = np.quantile(best_observed_per_seed, 0.75, axis=1)

        output = np.stack((cumulative_cost_per_seed.mean(axis=1), best_25, best_50, best_75),axis=-1)

        np.savetxt(f"results/quartiles/empirical_cost_aware/Empirical_VariableCost_{t}_{a}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

BudgetedMultiStepLookaheadEI LunarLander (3201, 16)


In [5]:
for a in acquisition_functions:
    for t in target_functions:
        config_and_metrics_per_seed = grouped_runs[a,t]

        cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0]).T 
        best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed if len(m['best observed'])>0]).T
            
        print(a, t, best_observed_per_seed.shape)

        mean = np.mean(best_observed_per_seed, axis=1)
        se = np.std(best_observed_per_seed, axis=1, ddof=1) / np.sqrt(best_observed_per_seed.shape[1])
        
        mean_plus_2se = mean + 2 * se
        mean_minus_2se = mean - 2 * se

        output = np.stack((cumulative_cost_per_seed.mean(axis=1), mean_minus_2se, mean, mean_plus_2se),axis=-1)

        np.savetxt(f"results/SE/empirical_cost_aware/Empirical_VariableCost_{t}_{a}.csv", output, header="cc, mean-2se, mean, mean+2se", delimiter=', ', comments='')

BudgetedMultiStepLookaheadEI LunarLander (3201, 16)


In [6]:
api = wandb.Api()

def load_empirical_variable_cost_runs(acq, tfn):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOPT", filters={
        "sweep": "5rtgsxvu", 
        "config.problem": tfn,
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [7]:
acquisition_functions = ['RandomSearch']
target_functions = ["LunarLander"]

In [8]:
grouped_runs = {(a,t): load_empirical_variable_cost_runs(a,t) for a in acquisition_functions for t in target_functions}

100%|██████████| 16/16 [00:14<00:00,  1.10it/s]


In [9]:
for a in acquisition_functions:
    for t in target_functions:
        config_and_metrics_per_seed = grouped_runs[a,t]

        cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0]).T 
        best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed if len(m['best observed'])>0]).T
            
        print(a, t, best_observed_per_seed.shape)

        mean = np.mean(best_observed_per_seed, axis=1)
        se = np.std(best_observed_per_seed, axis=1, ddof=1) / np.sqrt(best_observed_per_seed.shape[1])
        
        mean_plus_2se = mean + 2 * se
        mean_minus_2se = mean - 2 * se

        output = np.stack((cumulative_cost_per_seed.mean(axis=1), mean_minus_2se, mean, mean_plus_2se),axis=-1)

        np.savetxt(f"results/SE/empirical_cost_aware/Empirical_VariableCost_{t}_{a}.csv", output, header="cc, mean-2se, mean, mean+2se", delimiter=', ', comments='')

RandomSearch LunarLander (6001, 16)
